# Réplica del Paper: MP‑GCN para Reconocimiento de Actividades Grupales

Este cuaderno reproduce de forma simplificada las ideas principales del paper **"Skeleton‑based Group Activity Recognition via Spatial‑Temporal Panoramic Graph"**.  
Dado que el conjunto de datos original (Volleyball, NBA, Kinetics) y el código oficial requieren descargas externas, aquí implementamos una versión ligera que permite instalar las dependencias, construir las formas de entrada del alimentador (feeder), definir una versión mínima del modelo MP‑GCN y ejecutar un pase hacia adelante con datos sintéticos.  

> **Objetivos del cuaderno**
>
> 1. Instalar las dependencias esenciales (`torch`, `pyyaml`, `tqdm`, `fvcore`, `tensorboardX`).  
> 2. Construir un conjunto de datos sintético con forma \(X \in [C, T, V', M]\) donde \(C\) son canales (posición x,y, confianza), \(T\) es la longitud temporal, \(V'\) es el número de nodos (17 articulaciones humanas + *número de objetos*), y \(M\) es el número máximo de personas por secuencia.  
> 3. Definir matrices de adyacencia para las conexiones **intra‑persona** (topología del esqueleto humano), **persona↔objeto** (manos con objetos) e **inter‑persona** (pelvis con pelvis).  
> 4. Implementar una versión simplificada del MP‑GCN con cuatro *streams*: Joints (\(J\)), Bones (\(B\)), Joint Motion (\(JM\)) y Bone Motion (\(BM\)).  
> 5. Ejecutar un *forward* con datos sintéticos y mostrar las formas de tensores en cada etapa.

La intención es familiarizarse con el pipeline del artículo sin descargar grandes conjuntos de datos.


In [ ]:
# Instalación de dependencias
!pip -q install --upgrade pip
!pip -q install torch pyyaml tqdm tensorboardX fvcore


In [ ]:
import torch
import numpy as np

# Parámetros
C = 3            # canales: x, y, confidence
T = 48           # frames por ventana
num_joints = 17  # articulaciones humanas
n_objects = 1     # número de objetos
V = num_joints + n_objects  # nodos totales
M = 4            # número máximo de personas

# Datos sintéticos con forma [C, T, V, M]
synthetic_data = torch.rand(C, T, V, M)

print(f"Forma del tensor de entrada X: {synthetic_data.shape} (C,T,V',M)")

# Matrices de adyacencia
# A0: identidad
A0 = torch.eye(V)

# A_intra: conexiones del esqueleto y manos↔objeto
skeleton_edges = [
    (5,6),(5,7),(7,9),(6,8),(8,10),
    (11,12),(5,11),(6,12),(11,13),(13,15),(12,14),(14,16)
]
A_intra = torch.zeros(V, V)
for i, j in skeleton_edges:
    A_intra[i, j] = 1
    A_intra[j, i] = 1

# Conectar manos con objeto
obj_idx = V - 1
for hand in [9, 10]:
    A_intra[hand, obj_idx] = 1
    A_intra[obj_idx, hand] = 1

# A_inter: conexiones de pelvis (simplificado)
A_inter = torch.zeros(V, V)
A_inter[11, 11] = 1
A_inter[12, 12] = 1

print("""Matrices de adyacencia creadas:
 - A0: identidad
 - A_intra: conexiones esqueléticas y manos↔objeto
 - A_inter: conexiones de pelvis (simplificado)""")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleMPGCN(nn.Module):
    """Una versión mínima del modelo MP‑GCN. Aplica una convolución espacial sobre cada stream y luego fusiona los resultados."""
    def __init__(self, in_channels, out_channels, A0, A_intra, A_inter, num_classes):
        super().__init__()
        self.A0 = A0
        self.A_intra = A_intra
        self.A_inter = A_inter
        # Convoluciones para cada stream
        self.conv_J  = nn.Conv2d(in_channels, out_channels, kernel_size=(1,1))
        self.conv_B  = nn.Conv2d(in_channels, out_channels, kernel_size=(1,1))
        self.conv_JM = nn.Conv2d(in_channels, out_channels, kernel_size=(1,1))
        self.conv_BM = nn.Conv2d(in_channels, out_channels, kernel_size=(1,1))
        self.fc = nn.Linear(out_channels, num_classes)

    def forward(self, x):
        # x: [C,T,V,M]
        J = x
        # Bones: diferencias entre articulaciones conectadas
        B = torch.einsum('vw,ctwm->ctvm', self.A_intra, J) - J
        # Joint motion
        JM = J[:,1:] - J[:,:-1]
        JM = F.pad(JM, (0,0,0,0,0,1))
        # Bone motion
        B_shift = torch.einsum('vw,ctwm->ctvm', self.A_intra, J) - J
        BM = B_shift[:,1:] - B_shift[:,:-1]
        BM = F.pad(BM, (0,0,0,0,0,1))
        # Función auxiliar para aplicar conv y reducir
        def apply_conv(conv, stream):
            Cc, Tt, Vv, Mm = stream.shape
            stream = stream.permute(1,3,2,0).contiguous()  # [T,M,V,C]
            stream = stream.view(Tt, Mm*Vv, Cc).permute(2,0,1).unsqueeze(0)  # [1,C,T,VM]
            out = conv(stream)
            # Media sobre dimensiones temporales y nodos
            out = out.mean(dim=-1).mean(dim=-1)
            return out.squeeze(0)
        out_J  = apply_conv(self.conv_J,  J)
        out_B  = apply_conv(self.conv_B,  B)
        out_JM = apply_conv(self.conv_JM, JM)
        out_BM = apply_conv(self.conv_BM, BM)
        fused = out_J + out_B + out_JM + out_BM
        logits = self.fc(fused)
        return logits

# Inicializar y ejecutar el modelo
num_classes = 6
model = SimpleMPGCN(in_channels=C, out_channels=16, A0=A0, A_intra=A_intra, A_inter=A_inter, num_classes=num_classes)

# Ejecución de prueba
logits = model(synthetic_data)
print(f"Salida del modelo: logits de tamaño {logits.shape}")
print("Logits:", logits)


### Discusión y próximos pasos

En este cuaderno hemos:

- Instalado las dependencias necesarias para ejecutar un modelo de grafos en PyTorch.
- Definido la forma de entrada \([C,T,V',M]\) utilizada por MP‑GCN y generado un tensor sintético para simular datos.
- Construido matrices de adyacencia que representan las conexiones intra‑persona, persona↔objeto e inter‑persona.
- Implementado un modelo MP‑GCN básico que procesa cuatro *streams* (J, B, JM, BM) y genera una predicción para una de las seis etiquetas de escena.

Para replicar completamente los resultados del paper original sería necesario descargar y preparar los conjuntos de datos indicados en el repositorio oficial (Volleyball, NBA, Kinetics) y entrenar el modelo con configuraciones específicas. Este ejercicio sirve para familiarizarse con la estructura de entrada y el flujo de datos del modelo antes de trabajar con datos reales.
